# 06 · Manejo de caracteres y archivos

**Módulo I — Fundamentos**

## Objetivos:
- Comprender la codificación de caracteres y su importancia en datos médicos
- Resolver problemas comunes con tildes, ñ y caracteres especiales
- Manejar diferentes formatos de archivo y sistemas operativos
- Implementar soluciones robustas para intercambio de datos
- Aplicar buenas prácticas en sistemas hospitalarios

## Contexto Real - ¿Por qué es crucial en el sector salud?

En COMFENALCO y el sistema de salud colombiano:
- **Nombres de pacientes**: María José, José Ángel, Nuñez, Peña
- **Reportes a Supersalud**: Deben mantener caracteres originales
- **Intercambio con ADRES**: Sistemas con diferentes codificaciones
- **Bases de datos históricas**: Datos en Latin1, nuevos en UTF-8
- **Errores costosos**: Un nombre mal codificado = paciente no encontrado

## Índice:
1. Fundamentos de codificación de caracteres
2. Problemas comunes en datos médicos colombianos
3. Detección automática de encoding
4. Conversión entre formatos
5. Manejo robusto de archivos
6. Casos de error y soluciones
7. Implementación en sistemas EPS
8. Ejercicios con datos reales

## 1️⃣ Fundamentos de codificación de caracteres

### 🤔 Pregunta reflexiva para el grupo:
**¿Alguna vez han tenido problemas con nombres que aparecen como "José" en lugar de "José" en sus sistemas?**

### Contexto Real - Problema típico en EPS:
Un paciente se registra como "José Ángel Muñoz" pero en el sistema aparece como:
- Sistema A: "José Ángel Muñoz" 
- Sistema B: "Jos Ángel Muoz"
- Reporte Excel: "José Ã�ngel MuÃ±oz"

**Resultado**: El mismo paciente parece ser 3 personas diferentes.

In [ ]:
# Preparación del entorno
import pandas as pd
import numpy as np
from pathlib import Path
import chardet  # Para detección automática de encoding
import codecs
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas exitosamente")
print("Configurando entorno para manejo de caracteres especiales...")

# Crear estructura de carpetas
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data" / "encoding_examples"
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Carpeta de trabajo: {DATA_DIR}")
print("Listo para comenzar con ejercicios de codificación!")

In [ ]:
# Demostrar el problema con un ejemplo real
def demostrar_problema_encoding():
    """
    Demuestra cómo los mismos caracteres se ven diferentes según la codificación.
    """
    # Nombres típicos colombianos con caracteres especiales
    nombres_colombia = [
        "María José Pérez",
        "José Ángel Muñoz", 
        "Sofía Hernández",
        "Nicolás Peña",
        "Andrés Núñez"
    ]
    
    print("DEMOSTRACIÓN: El mismo nombre en diferentes codificaciones")
    print("="*60)
    
    nombre_ejemplo = "José Ángel Muñoz"
    print(f"Nombre original: {nombre_ejemplo}")
    print()
    
    # Mostrar cómo se ve en diferentes encodings
    encodings_comunes = ['utf-8', 'latin1', 'cp1252', 'ascii']
    
    for encoding in encodings_comunes:
        try:
            # Codificar y decodificar para simular el problema
            bytes_encoded = nombre_ejemplo.encode('utf-8')
            
            if encoding == 'ascii':
                # ASCII no puede manejar caracteres especiales
                resultado = "ERROR: No se pueden procesar caracteres especiales"
            else:
                # Decodificar con encoding incorrecto
                resultado = bytes_encoded.decode(encoding, errors='replace')
            
            print(f"{encoding.upper():<8}: {resultado}")
            
        except Exception as e:
            print(f"{encoding.upper():<8}: ERROR - {str(e)}")
    
    return nombres_colombia

nombres_test = demostrar_problema_encoding()

# CHECKPOINT 1
print("\n" + "="*50)
print("✅ CHECKPOINT 1: ¿Pueden ver las diferencias en los nombres?")
print("👨‍🏫 Instructor: Verifiquemos que todos vean los resultados")
print("🤔 ¿Cuál creen que es la codificación correcta para nombres colombianos?")
print("="*50)

## 2️⃣ Problemas comunes en datos médicos colombianos

### Contexto Real - Casos que veremos en COMFENALCO:
1. **Importar CSV desde Excel**: Latin1 vs UTF-8
2. **Reportes de MinSalud**: Diferentes sistemas, diferentes encodings
3. **Bases de datos legacy**: Datos históricos en ASCII extendido
4. **APIs modernas**: UTF-8 por defecto
5. **Archivos de intercambio**: RIPS, BDUA con caracteres especiales

### 🤔 Pregunta reflexiva:
**¿Han notado que algunos reportes muestran caracteres extraños cuando los abren en Excel?**

In [ ]:
# Crear dataset con problemas típicos de encoding
def crear_datos_problema_encoding():
    """
    Crea archivos con diferentes encodings para simular problemas reales.
    """
    # Datos típicos de una EPS con caracteres problemáticos
    datos_eps = {
        'id_afiliado': [1001, 1002, 1003, 1004, 1005],
        'nombres': [
            'María José',
            'José Ángel', 
            'Ana Sofía',
            'Luis Hernán',
            'Nicolás'
        ],
        'apellidos': [
            'Pérez García',
            'Muñoz López',
            'Hernández Peña',
            'Núñez Vásquez',
            'Gómez Sánchez'
        ],
        'municipio': [
            'Bogotá D.C.',
            'Medellín',
            'Cali',
            'Pereira', 
            'Manizales'
        ],
        'diagnostico': [
            'Hipertensión arterial',
            'Diabetes mellitus',
            'Cefalea tensional',
            'Lumbalgia mecánica',
            'Rinitis alérgica'
        ]
    }
    
    df_eps = pd.DataFrame(datos_eps)
    
    print("Creando archivos con diferentes encodings...")
    print("(Esto simula archivos que llegan de diferentes sistemas)")
    
    # Archivo 1: UTF-8 (correcto)
    archivo_utf8 = DATA_DIR / "afiliados_utf8.csv"
    df_eps.to_csv(archivo_utf8, index=False, encoding='utf-8')
    
    # Archivo 2: Latin1 (común en sistemas legacy)
    archivo_latin1 = DATA_DIR / "afiliados_latin1.csv"
    df_eps.to_csv(archivo_latin1, index=False, encoding='latin1')
    
    # Archivo 3: CP1252 (Windows español)
    archivo_cp1252 = DATA_DIR / "afiliados_cp1252.csv"
    df_eps.to_csv(archivo_cp1252, index=False, encoding='cp1252')
    
    archivos_creados = [
        (archivo_utf8, 'utf-8', 'Sistema moderno (recomendado)'),
        (archivo_latin1, 'latin1', 'Sistema legacy hospitalario'),
        (archivo_cp1252, 'cp1252', 'Excel Windows español')
    ]
    
    print("\nArchivos creados:")
    for archivo, encoding, descripcion in archivos_creados:
        tamaño = archivo.stat().st_size / 1024
        print(f"  ✅ {archivo.name:<25} ({encoding:<8}) - {descripcion}")
    
    return df_eps, archivos_creados

df_original, archivos_test = crear_datos_problema_encoding()
print(f"\nDatos originales:")
display(df_original)

In [ ]:
# Demostrar qué pasa al abrir con encoding incorrecto
def demostrar_lectura_incorrecta():
    """
    Muestra el ERROR común: abrir archivo con encoding incorrecto.
    """
    print("DEMOSTRACIÓN: ¿Qué pasa al leer con encoding incorrecto?")
    print("(Esto es lo que pasa cuando Excel abre un archivo UTF-8)")
    print("="*60)
    
    archivo_utf8 = DATA_DIR / "afiliados_utf8.csv"
    
    # Lectura correcta (UTF-8)
    print("1. LECTURA CORRECTA (UTF-8):")
    df_correcto = pd.read_csv(archivo_utf8, encoding='utf-8')
    print("Nombres:", list(df_correcto['nombres']))
    
    # Lectura incorrecta (Latin1)
    print("\n2. LECTURA INCORRECTA (Latin1):")
    print("(Simula abrir archivo UTF-8 como si fuera Latin1)")
    try:
        df_incorrecto = pd.read_csv(archivo_utf8, encoding='latin1')
        print("Nombres:", list(df_incorrecto['nombres']))
        print("👀 ¿Notan la diferencia? Los acentos se ven raros.")
    except Exception as e:
        print(f"Error: {e}")
    
    return df_correcto, df_incorrecto

df_bien, df_mal = demostrar_lectura_incorrecta()

# CHECKPOINT 2
print("\n" + "="*50)
print("✅ CHECKPOINT 2: Comparación de lecturas")
print("👨‍🏫 ¿Todos pueden ver la diferencia en los nombres?")
print("🤔 En su trabajo, ¿han visto nombres con caracteres extraños?")
print("💡 Esto explica por qué a veces los reportes se ven mal en Excel")
print("="*50)

## 3️⃣ Detección automática de encoding

### Contexto Real - Situación típica en COMFENALCO:
**Llega un archivo de otra IPS con nombres que no se ven bien. ¿Cómo saber qué encoding usar?**

### 🔧 Solución práctica:
Usar la librería `chardet` para detectar automáticamente el encoding.

### 🤔 Pregunta reflexiva:
**¿Prefieren adivinar el encoding o que el sistema lo detecte automáticamente?**

In [ ]:
# Función para detectar encoding automáticamente
def detectar_encoding_archivo(ruta_archivo):
    """
    Detecta automáticamente la codificación de un archivo.
    Útil cuando llegan archivos de sistemas externos.
    """
    print(f"\nAnalizando archivo: {ruta_archivo.name}")
    
    # Leer una muestra del archivo
    with open(ruta_archivo, 'rb') as archivo:
        muestra = archivo.read(10000)  # Primeros 10KB
    
    # Detectar encoding
    resultado = chardet.detect(muestra)
    
    print(f"  Encoding detectado: {resultado['encoding']}")
    print(f"  Confianza: {resultado['confidence']*100:.1f}%")
    print(f"  Lenguaje: {resultado.get('language', 'No detectado')}")
    
    return resultado['encoding'], resultado['confidence']

# Detectar encoding de todos nuestros archivos de prueba
print("DETECCIÓN AUTOMÁTICA DE ENCODING")
print("="*50)
print("(Esto es lo que haríamos con archivos de origen desconocido)")

resultados_deteccion = {}
for archivo_path, encoding_real, descripcion in archivos_test:
    encoding_detectado, confianza = detectar_encoding_archivo(archivo_path)
    resultados_deteccion[archivo_path.name] = {
        'real': encoding_real,
        'detectado': encoding_detectado,
        'confianza': confianza
    }
    print(f"  ✅ Encoding real: {encoding_real} | Detectado: {encoding_detectado}")
    print(f"      Coincidencia: {'SÍ' if encoding_real == encoding_detectado else 'NO'}")

In [ ]:
# Función robusta para leer archivos con encoding automático
def leer_csv_robusto(ruta_archivo, mostrar_proceso=True):
    """
    Lee un archivo CSV detectando automáticamente el encoding.
    Esta función es la que usaríamos en producción en COMFENALCO.
    """
    if mostrar_proceso:
        print(f"\n📂 Abriendo archivo: {ruta_archivo.name}")
    
    # Paso 1: Detectar encoding
    with open(ruta_archivo, 'rb') as archivo:
        muestra = archivo.read(10000)
    
    deteccion = chardet.detect(muestra)
    encoding_detectado = deteccion['encoding']
    confianza = deteccion['confidence']
    
    if mostrar_proceso:
        print(f"   🔍 Encoding detectado: {encoding_detectado} ({confianza*100:.1f}% confianza)")
    
    # Paso 2: Leer con encoding detectado
    try:
        df = pd.read_csv(ruta_archivo, encoding=encoding_detectado)
        if mostrar_proceso:
            print(f"   ✅ Archivo leído exitosamente: {df.shape[0]} filas, {df.shape[1]} columnas")
        return df, encoding_detectado
        
    except Exception as e:
        if mostrar_proceso:
            print(f"   ❌ Error con encoding detectado: {e}")
            print(f"   🔄 Intentando con UTF-8...")
        
        # Paso 3: Fallback a UTF-8
        try:
            df = pd.read_csv(ruta_archivo, encoding='utf-8')
            if mostrar_proceso:
                print(f"   ✅ Leído con UTF-8: {df.shape[0]} filas, {df.shape[1]} columnas")
            return df, 'utf-8'
        except:
            # Paso 4: Fallback final con errors='replace'
            if mostrar_proceso:
                print(f"   🔄 Último intento con reemplazo de caracteres...")
            df = pd.read_csv(ruta_archivo, encoding='utf-8', errors='replace')
            if mostrar_proceso:
                print(f"   ⚠️  Leído con reemplazo de caracteres: {df.shape[0]} filas")
            return df, 'utf-8-replace'

# Probar la función robusta con todos nuestros archivos
print("\nPROBANDO FUNCIÓN ROBUSTA DE LECTURA")
print("="*50)
print("(Esta sería nuestra función estándar para COMFENALCO)")

dataframes_leidos = []
for archivo_path, encoding_real, descripcion in archivos_test:
    df, encoding_usado = leer_csv_robusto(archivo_path)
    dataframes_leidos.append((archivo_path.name, df, encoding_usado))
    
    # Mostrar una muestra de los nombres para verificar
    print(f"   👤 Muestra de nombres: {df['nombres'].iloc[0]}, {df['nombres'].iloc[1]}")

## 4️⃣ Conversión entre formatos

### Contexto Real - Caso práctico COMFENALCO:
**Escenario**: Tenemos archivos históricos en Latin1 y necesitamos migrarlos a UTF-8 para el nuevo sistema.

### 🤔 Pregunta reflexiva:
**¿Han tenido que migrar datos de un sistema viejo a uno nuevo? ¿Qué problemas encontraron?**

### ⚠️ Caso de error común:
**Error típico**: Convertir directamente sin verificar, perdiendo caracteres especiales.

In [ ]:
# CHECKPOINT 3
print("✅ CHECKPOINT 3: Verificación antes de conversión")
print("👨‍🏫 ¿Todos pudieron leer correctamente los tres archivos?")
print("🤔 ¿Notan alguna diferencia en la calidad de los nombres?")
print("💼 Conexión EPS: Así procesaríamos archivos de diferentes IPS")
print("="*60)

# Función para conversión segura de encodings
def convertir_encoding_seguro(archivo_origen, archivo_destino, encoding_destino='utf-8'):
    """
    Convierte un archivo de un encoding a otro de forma segura.
    Incluye validación y reporte de problemas.
    """
    print(f"\n🔄 CONVERSIÓN DE ENCODING")
    print(f"   Origen: {archivo_origen.name}")
    print(f"   Destino: {archivo_destino.name}")
    print(f"   Encoding objetivo: {encoding_destino}")
    
    # Paso 1: Leer archivo origen con detección automática
    df_origen, encoding_origen = leer_csv_robusto(archivo_origen, mostrar_proceso=False)
    print(f"   📖 Encoding origen detectado: {encoding_origen}")
    
    # Paso 2: Verificar que no hay pérdida de datos
    caracteres_especiales = ['á', 'é', 'í', 'ó', 'ú', 'ñ', 'Á', 'É', 'Í', 'Ó', 'Ú', 'Ñ']
    
    texto_completo = ' '.join(df_origen.astype(str).values.flatten())
    especiales_encontrados = [c for c in caracteres_especiales if c in texto_completo]
    
    print(f"   🔍 Caracteres especiales encontrados: {especiales_encontrados}")
    
    # Paso 3: Realizar conversión
    try:
        df_origen.to_csv(archivo_destino, index=False, encoding=encoding_destino)
        print(f"   ✅ Conversión exitosa")
        
        # Paso 4: Verificar conversión leyendo el archivo convertido
        df_verificacion = pd.read_csv(archivo_destino, encoding=encoding_destino)
        texto_verificacion = ' '.join(df_verificacion.astype(str).values.flatten())
        especiales_despues = [c for c in caracteres_especiales if c in texto_verificacion]
        
        if set(especiales_encontrados) == set(especiales_despues):
            print(f"   ✅ Verificación: No se perdieron caracteres especiales")
            return True
        else:
            perdidos = set(especiales_encontrados) - set(especiales_despues)
            print(f"   ⚠️  Advertencia: Se perdieron caracteres: {perdidos}")
            return False
            
    except Exception as e:
        print(f"   ❌ Error en conversión: {e}")
        return False

# Realizar conversiones de ejemplo
print("EJEMPLO DE MIGRACIÓN: Latin1 -> UTF-8")
print("(Caso típico: Sistema legacy -> Sistema moderno)")

archivo_latin1 = DATA_DIR / "afiliados_latin1.csv"
archivo_convertido = DATA_DIR / "afiliados_latin1_to_utf8.csv"

exito = convertir_encoding_seguro(archivo_latin1, archivo_convertido, 'utf-8')

if exito:
    print("\n🎉 ¡Migración exitosa!")
    print("💼 En COMFENALCO: Así migraríamos datos históricos al nuevo sistema")
else:
    print("\n⚠️  Se requiere revisión manual")

## 5️⃣ Manejo robusto de archivos

### Contexto Real - Operación diaria en COMFENALCO:
**Situación**: Llegan 20 archivos diarios de diferentes fuentes:
- Laboratorios (Latin1)
- Farmacias (UTF-8) 
- Hospitales (CP1252)
- MinSalud (UTF-8)
- Supersalud (Variable)

### 🤔 Pregunta reflexiva:
**¿Cómo procesarían ustedes 20 archivos diferentes cada día sin saber su encoding?**

### 💡 Solución: Pipeline automático robusto

In [ ]:
# Pipeline robusto para procesamiento masivo
def procesar_archivos_masivo(carpeta_archivos, carpeta_salida):
    """
    Procesa múltiples archivos automáticamente.
    Esta sería la función que ejecutaríamos diariamente en COMFENALCO.
    """
    carpeta_salida.mkdir(exist_ok=True)
    
    archivos_csv = list(carpeta_archivos.glob("*.csv"))
    
    print(f"\n🏭 PROCESAMIENTO MASIVO DE ARCHIVOS")
    print(f"📁 Carpeta origen: {carpeta_archivos}")
    print(f"📁 Carpeta destino: {carpeta_salida}")
    print(f"📊 Archivos encontrados: {len(archivos_csv)}")
    print("="*50)
    
    resultados = {
        'exitosos': [],
        'errores': [],
        'advertencias': []
    }
    
    for i, archivo in enumerate(archivos_csv, 1):
        print(f"\n[{i}/{len(archivos_csv)}] Procesando: {archivo.name}")
        
        try:
            # Leer con detección automática
            df, encoding_usado = leer_csv_robusto(archivo, mostrar_proceso=False)
            
            # Validar datos básicos
            filas_vacias = df.isnull().all(axis=1).sum()
            cols_vacias = df.isnull().all(axis=0).sum()
            
            print(f"   📋 {df.shape[0]} filas, {df.shape[1]} columnas")
            print(f"   🔤 Encoding: {encoding_usado}")
            
            if filas_vacias > 0:
                print(f"   ⚠️  {filas_vacias} filas completamente vacías")
                resultados['advertencias'].append(f"{archivo.name}: {filas_vacias} filas vacías")
            
            if cols_vacias > 0:
                print(f"   ⚠️  {cols_vacias} columnas completamente vacías")
                resultados['advertencias'].append(f"{archivo.name}: {cols_vacias} columnas vacías")
            
            # Guardar normalizado en UTF-8
            archivo_salida = carpeta_salida / f"normalizado_{archivo.name}"
            df.to_csv(archivo_salida, index=False, encoding='utf-8')
            
            print(f"   ✅ Procesado y guardado como: {archivo_salida.name}")
            resultados['exitosos'].append(archivo.name)
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            resultados['errores'].append(f"{archivo.name}: {str(e)}")
    
    # Reporte final
    print(f"\n📊 REPORTE FINAL:")
    print(f"   ✅ Exitosos: {len(resultados['exitosos'])}")
    print(f"   ⚠️  Con advertencias: {len(resultados['advertencias'])}")
    print(f"   ❌ Con errores: {len(resultados['errores'])}")
    
    if resultados['errores']:
        print(f"\n❌ ERRORES ENCONTRADOS:")
        for error in resultados['errores']:
            print(f"   • {error}")
    
    return resultados

# Crear carpeta de salida
carpeta_normalizados = DATA_DIR / "normalizados"

# Procesar todos los archivos de ejemplo
resultado_proceso = procesar_archivos_masivo(DATA_DIR, carpeta_normalizados)

print("\n💼 Aplicación en COMFENALCO:")
print("   - Este proceso se ejecutaría automáticamente cada día")
print("   - Los archivos normalizados se cargarían a la base de datos")
print("   - Los errores se reportarían al equipo técnico")

## 6️⃣ Casos de error y soluciones

### ⚠️ Errores más comunes en sistemas de salud:

1. **UnicodeDecodeError**: Archivo con encoding incorrecto
2. **Caracteres de reemplazo**: � � � en lugar de acentos
3. **Pérdida de información**: Nombres truncados
4. **Inconsistencia**: Mismo paciente con diferentes nombres

### 🤔 Pregunta reflexiva:
**¿Cuál de estos errores consideran más grave para la operación de una EPS?**

In [ ]:
# Simulador de errores comunes y sus soluciones
def simular_errores_comunes():
    """
    Simula y resuelve errores típicos de encoding en sistemas médicos.
    """
    print("🚨 SIMULADOR DE ERRORES COMUNES")
    print("================================")
    
    # Error 1: UnicodeDecodeError
    print("\n1. ERROR: UnicodeDecodeError")
    print("   Situación: Intentar leer archivo Latin1 como UTF-8")
    
    archivo_latin1 = DATA_DIR / "afiliados_latin1.csv"
    
    print("   ❌ Forma incorrecta:")
    try:
        # Esto causará problemas
        df_error = pd.read_csv(archivo_latin1, encoding='utf-8')
        print("   (No debería llegar aquí)")
    except UnicodeDecodeError as e:
        print(f"      UnicodeDecodeError: {str(e)[:100]}...")
    
    print("\n   ✅ Solución correcta:")
    df_correcto, encoding = leer_csv_robusto(archivo_latin1, mostrar_proceso=False)
    print(f"      Archivo leído correctamente con {encoding}")
    print(f"      Primer nombre: {df_correcto['nombres'].iloc[0]}")
    
    # Error 2: Caracteres de reemplazo
    print("\n2. ERROR: Caracteres de reemplazo (�)")
    print("   Situación: Forzar lectura con errors='replace'")
    
    print("   ❌ Resultado problemático:")
    try:
        df_reemplazos = pd.read_csv(archivo_latin1, encoding='utf-8', errors='replace')
        print(f"      Nombre con reemplazos: {df_reemplazos['nombres'].iloc[1]}")
        print("      👀 ¿Notan los caracteres extraños?")
    except Exception as e:
        print(f"      Error: {e}")
    
    print("\n   ✅ Solución: Usar encoding correcto desde el inicio")
    df_sin_problemas = pd.read_csv(archivo_latin1, encoding='latin1')
    print(f"      Nombre correcto: {df_sin_problemas['nombres'].iloc[1]}")
    
    # Error 3: Detección incorrecta
    print("\n3. PROBLEMA: Detección automática fallida")
    print("   Situación: chardet detecta mal el encoding")
    
    # Crear archivo problemático
    texto_problematico = "José,María,Ángel\n25,30,28\n"
    archivo_problema = DATA_DIR / "problema_deteccion.csv"
    
    with open(archivo_problema, 'w', encoding='latin1') as f:
        f.write(texto_problematico)
    
    # Detectar (puede fallar)
    with open(archivo_problema, 'rb') as f:
        muestra = f.read()
    deteccion = chardet.detect(muestra)
    
    print(f"   🔍 chardet detectó: {deteccion['encoding']} ({deteccion['confidence']*100:.1f}% confianza)")
    print(f"   📝 Encoding real: latin1")
    
    if deteccion['confidence'] < 0.7:
        print("   ⚠️  Baja confianza - requiere verificación manual")
    
    print("\n   ✅ Solución: Pipeline con múltiples intentos")
    print("      1. Usar detección automática si confianza > 70%")
    print("      2. Si falla, probar UTF-8")
    print("      3. Si falla, probar Latin1")
    print("      4. Como último recurso, usar 'replace'")

simular_errores_comunes()

# CHECKPOINT 4 - FINAL
print("\n" + "="*60)
print("✅ CHECKPOINT FINAL: Revisión completa")
print("👨‍🏫 ¿Todos comprenden los 3 tipos de errores mostrados?")
print("🤔 ¿Cuál creen que es más común en su trabajo diario?")
print("💼 En COMFENALCO: Estos errores pueden afectar la facturación")
print("🎯 Objetivo alcanzado: Saben detectar y corregir problemas de encoding")
print("="*60)

## 7️⃣ Implementación en sistemas EPS

### Contexto Real - Implementación práctica en COMFENALCO:

#### 🏥 Casos de uso específicos:
1. **RIPS (Reportes integrales)**: Caracteres especiales en diagnósticos
2. **BDUA (Base única de afiliados)**: Nombres con tildes y ñ
3. **Intercambio con IPS**: Archivos de diferentes sistemas
4. **Reportes Supersalud**: Formato estricto UTF-8
5. **Facturación**: Nombres exactos para validación

### 🤔 Pregunta reflexiva final:
**¿Cómo implementarían esto en su flujo de trabajo actual?**

In [ ]:
# Clase para implementación empresarial
class ManejadorEncodingEPS:
    """
    Clase para manejo robusto de encoding en sistemas EPS.
    Lista para usar en producción en COMFENALCO.
    """
    
    def __init__(self, carpeta_logs=None):
        self.encodings_comunes = ['utf-8', 'latin1', 'cp1252', 'ascii']
        self.carpeta_logs = carpeta_logs or Path('logs')
        self.carpeta_logs.mkdir(exist_ok=True)
        
        # Configurar logging
        self.log_procesamiento = []
    
    def leer_archivo_seguro(self, ruta_archivo):
        """Lectura robusta con logging detallado."""
        inicio = datetime.now()
        
        try:
            # Detección automática
            with open(ruta_archivo, 'rb') as f:
                muestra = f.read(10000)
            
            deteccion = chardet.detect(muestra)
            encoding_detectado = deteccion['encoding']
            confianza = deteccion['confidence']
            
            # Intentar lectura
            df = pd.read_csv(ruta_archivo, encoding=encoding_detectado)
            
            # Log exitoso
            log_entry = {
                'archivo': ruta_archivo.name,
                'timestamp': inicio.isoformat(),
                'encoding_detectado': encoding_detectado,
                'confianza': confianza,
                'filas': df.shape[0],
                'columnas': df.shape[1],
                'estado': 'EXITOSO',
                'tiempo_procesamiento': (datetime.now() - inicio).total_seconds()
            }
            
            self.log_procesamiento.append(log_entry)
            return df, log_entry
            
        except Exception as e:
            # Log de error
            log_entry = {
                'archivo': ruta_archivo.name,
                'timestamp': inicio.isoformat(),
                'error': str(e),
                'estado': 'ERROR',
                'tiempo_procesamiento': (datetime.now() - inicio).total_seconds()
            }
            
            self.log_procesamiento.append(log_entry)
            raise
    
    def generar_reporte(self):
        """Genera reporte de procesamiento para auditoría."""
        if not self.log_procesamiento:
            return "No hay datos de procesamiento"
        
        df_log = pd.DataFrame(self.log_procesamiento)
        
        reporte = {
            'total_archivos': len(df_log),
            'exitosos': len(df_log[df_log['estado'] == 'EXITOSO']),
            'errores': len(df_log[df_log['estado'] == 'ERROR']),
            'tiempo_total': df_log['tiempo_procesamiento'].sum(),
            'encodings_detectados': df_log['encoding_detectado'].value_counts().to_dict() if 'encoding_detectado' in df_log.columns else {}
        }
        
        return reporte

# Demostrar uso empresarial
print("💼 IMPLEMENTACIÓN PARA COMFENALCO")
print("=================================")

manejador = ManejadorEncodingEPS()

# Procesar archivos con logging
archivos_ejemplo = list(DATA_DIR.glob("afiliados_*.csv"))

print(f"Procesando {len(archivos_ejemplo)} archivos con logging empresarial...\n")

for archivo in archivos_ejemplo:
    try:
        df, log = manejador.leer_archivo_seguro(archivo)
        print(f"✅ {archivo.name}: {log['filas']} filas, encoding {log['encoding_detectado']}")
    except Exception as e:
        print(f"❌ {archivo.name}: Error - {str(e)[:50]}...")

# Generar reporte final
reporte = manejador.generar_reporte()
print(f"\n📊 REPORTE DE PROCESAMIENTO:")
for key, value in reporte.items():
    print(f"   {key}: {value}")

print("\n🎯 BENEFICIOS PARA COMFENALCO:")
print("   ✅ Procesamiento automático y confiable")
print("   📋 Logs detallados para auditoría")
print("   🔍 Detección temprana de problemas")
print("   📈 Métricas de calidad de datos")
print("   ⚡ Reducción de tiempo manual")

## 8️⃣ Ejercicios prácticos con datos reales

### 🏃‍♂️ Ejercicio 1: Detección y corrección
**Contexto**: Han llegado 3 archivos de diferentes IPS con problemas de encoding.
1. Detecta automáticamente el encoding de cada archivo
2. Lee los datos correctamente
3. Identifica qué nombres tienen problemas
4. Genera un reporte de calidad

### 🏃‍♀️ Ejercicio 2: Migración masiva
**Contexto**: Migrar 50 archivos históricos de Latin1 a UTF-8.
1. Crea una función de migración por lotes
2. Incluye validación antes y después
3. Genera reporte de archivos problemáticos
4. Implementa rollback en caso de errores

### 🏃‍♂️ Ejercicio 3: Pipeline de producción
**Contexto**: Crear sistema automático para COMFENALCO.
1. Pipeline que procese archivos diarios automáticamente
2. Sistema de alertas para archivos problemáticos
3. Dashboard con métricas de calidad
4. Integración con base de datos

### 🏃‍♀️ Ejercicio 4: Caso real RIPS
**Contexto**: Procesar archivos RIPS con diagnósticos en español.
1. Manejar códigos CIE-10 con descripción
2. Validar que no se pierdan caracteres especiales
3. Generar archivo para Supersalud en UTF-8
4. Crear reporte de validación

In [ ]:
# Plantilla para resolver ejercicios
def plantilla_ejercicios():
    """
    Espacio para que los participantes resuelvan los ejercicios.
    """
    print("✍️ ESPACIO PARA EJERCICIOS PRÁCTICOS")
    print("====================================")
    
    print("\n🏃‍♂️ Ejercicio 1: Detección y corrección")
    print("# Su código aquí:")
    print("# archivos_problema = [archivo1, archivo2, archivo3]")
    print("# for archivo in archivos_problema:")
    print("#     encoding, confianza = detectar_encoding_archivo(archivo)")
    
    print("\n🏃‍♀️ Ejercicio 2: Migración masiva")
    print("# Su código aquí:")
    print("# def migrar_lote(archivos_origen, carpeta_destino):")
    print("#     # Implementar migración con validación")
    
    print("\n🏃‍♂️ Ejercicio 3: Pipeline de producción")
    print("# Su código aquí:")
    print("# class PipelineComfenalco:")
    print("#     def procesar_diario(self):")
    print("#         # Implementar procesamiento automático")
    
    print("\n🏃‍♀️ Ejercicio 4: Caso real RIPS")
    print("# Su código aquí:")
    print("# def procesar_rips(archivo_entrada, archivo_salida):")
    print("#     # Validar diagnósticos CIE-10")
    print("#     # Mantener caracteres especiales")
    
    # Checklist de verificación
    checklist = [
        "☐ Ejercicio 1: Detección automática implementada",
        "☐ Ejercicio 2: Migración masiva con validación",
        "☐ Ejercicio 3: Pipeline automático funcional",
        "☐ Ejercicio 4: Procesamiento RIPS validado"
    ]
    
    print(f"\n✅ CHECKLIST DE VERIFICACIÓN:")
    for item in checklist:
        print(f"   {item}")
    
    print(f"\n🎯 CRITERIOS DE ÉXITO:")
    print(f"   • No se pierden caracteres especiales")
    print(f"   • Manejo robusto de errores")
    print(f"   • Logging adecuado para auditoría")
    print(f"   • Aplicabilidad real en COMFENALCO")

plantilla_ejercicios()

## ✅ Resumen del Notebook

### 🎯 Objetivos alcanzados:
1. ✅ **Comprensión profunda** de codificación de caracteres
2. ✅ **Resolución práctica** de problemas con nombres colombianos
3. ✅ **Herramientas robustas** para detección automática
4. ✅ **Pipeline empresarial** listo para producción
5. ✅ **Casos de error** y sus soluciones

### 🏥 Aplicación directa en COMFENALCO:
- **Procesamiento RIPS**: Sin pérdida de caracteres especiales
- **Intercambio con IPS**: Manejo automático de diferentes encodings
- **Reportes Supersalud**: Formato UTF-8 garantizado
- **Migración de datos**: Sistema legacy → moderno
- **Validación diaria**: Detección temprana de problemas

### 🔧 Herramientas implementadas:
- Detector automático de encoding
- Lector robusto de archivos
- Convertidor seguro entre formatos
- Pipeline de procesamiento masivo
- Sistema de logging empresarial

### ⚠️ Errores críticos prevenidos:
- **UnicodeDecodeError**: Archivos no legibles
- **Caracteres perdidos**: Nombres truncados
- **Datos corruptos**: Información irrecuperable
- **Inconsistencias**: Mismo paciente, nombres diferentes

### 🚀 Próximos pasos:
- **Módulo II:** Conexión avanzada con bases de datos
- **Módulo III:** Validación y limpieza de datos médicos
- **Módulo IV:** Automatización de reportes

---

**💡 Mensaje clave**: En el sector salud, un nombre mal codificado puede significar un paciente no encontrado. Con estas herramientas, COMFENALCO puede procesar datos de cualquier fuente sin perder información crítica.